In [1]:
import sys
sys.path.append('/home') 
import arpespythontools as arp 
import matplotlib.pyplot as plt
import numpy as np
import time
%matplotlib inline

In [2]:
url = 'https://pranabdas.github.io/drive/datasets/arpes/sample_map_data.zip'
data, energy, theta, phi = arp.load_ses_map(url)

In [4]:
t0 = time.perf_counter()
data_k, e_bin, kx, ky = arp.k_conv3d(data, energy, theta, phi, 16.67)
t_final = time.perf_counter()
print("The program took", t_final - t0, "second(s).")

The program took 97.2411813929998 second(s).


In [5]:
def interp(data_slice, energy_val, theta, phi, kx, ky):
    from scipy import interpolate
    import numpy as np
    
    theta_temp = np.arcsin(kx/(0.512*np.sqrt(energy_val)))*180/np.pi
    phi_temp = np.linspace(0, 0, len(phi))
    theta_grid, phi_grid = np.meshgrid(theta_temp, phi_temp)

    for jj in range(np.shape(data_slice)[1]):
        for kk in range(np.shape(data_slice)[0]):
            phi_grid[kk, jj] = np.arcsin(ky[kk]/(0.512*np.sqrt(energy_val)\
                    *np.cos(theta_grid[0, jj]*np.pi/180)))*180/np.pi

    func = interpolate.RectBivariateSpline(phi, theta, data_slice)
    data_k = func.ev(phi_grid, theta_grid)

    # nan wrapping for outside interpolation range data points
    mask_theta = (theta_grid < theta[0]) | (theta_grid > theta[-1])
    mask_phi = (phi_grid < phi[0]) | (phi_grid > phi[-1])
    mask = mask_theta | mask_phi
    data_k[mask] = np.nan
    return data_k
        
def k_conv3d_mp(data, energy, theta, phi, fermi_energy):
    '''
    data_k, e_bin, kx, ky = k_conv3d(data, energy, theta, phi, fermi_energy)
    '''
    import numpy as np
    from multiprocessing import Pool
    from itertools import repeat
    import time
    t0 = time.perf_counter()

    data = np.transpose(data, (0, 2, 1))
    # Transpose the data, 0 -> energy, 1 -> phi, 2 -> theta
    kx_min = 0.512 * np.sqrt(energy[-1]) * np.sin(theta[0]*np.pi/180)
    kx_max = 0.512 * np.sqrt(energy[-1]) * np.sin(theta[len(theta)-1]*np.pi/180)
    kx = np.linspace(kx_min, kx_max, len(theta))

    ky_min = 0.512 * np.sqrt(energy[-1]) * np.sin(phi[0]*np.pi/180)
    ky_max = 0.512 * np.sqrt(energy[-1]) * np.sin(phi[len(phi)-1]*np.pi/180)
    ky = np.linspace(ky_min, ky_max, len(phi))
  
    pool = Pool()
    input_array_tuple = zip(data, energy, repeat(theta), repeat(phi), repeat(kx), repeat(ky))
    data_k = pool.starmap(interp, input_array_tuple)
    pool.close()
    data_k = np.array(data_k)
    data_k = np.transpose(data_k, (0, 2, 1)) # Transpose back to original order

    e_bin = fermi_energy - energy
    
    t_final = time.perf_counter()
    print("The program took", t_final - t0, "second(s).")
    return data_k, e_bin, kx, ky

In [6]:
data_k_mp, e_bin, kx, ky = k_conv3d_mp(data, energy, theta, phi, 16.67)

The program took 42.235067427000104 second(s).


In [7]:
np.array_equal(data_k, data_k_mp, equal_nan=True)

True

In [8]:
# here we change the ordering of meshgrid
# the result is almost equal, but not exact equal to previous program
def interp(data_slice, energy_val, theta, phi, kx, ky):
    from scipy import interpolate
    import numpy as np
    
    theta_temp = np.arcsin(kx/(0.512*np.sqrt(energy_val)))*180/np.pi
    phi_temp = np.linspace(0, 0, len(phi))
    theta_grid, phi_grid = np.meshgrid(theta_temp, phi_temp, indexing='ij')
    
    for jj in range(len(theta)):
        for kk in range(len(phi)):
            phi_grid[jj, kk] = np.arcsin(ky[kk]/(0.512*np.sqrt(energy_val)\
                    *np.cos(theta_grid[jj, 0]*np.pi/180)))*180/np.pi

    func = interpolate.RectBivariateSpline(theta, phi, data_slice)
    data_k = func.ev(theta_grid, phi_grid)

    # nan wrapping for outside interpolation range data points
    mask_theta = (theta_grid < theta[0]) | (theta_grid > theta[-1])
    mask_phi = (phi_grid < phi[0]) | (phi_grid > phi[-1])
    mask = mask_theta | mask_phi
    data_k[mask] = np.nan
    return data_k
        
def k_conv3d_mp_alt(data, energy, theta, phi, fermi_energy):
    '''
    data_k, e_bin, kx, ky = k_conv3d(data, energy, theta, phi, fermi_energy)
    '''
    import numpy as np
    from multiprocessing import Pool
    from itertools import repeat
    import time
    t0 = time.perf_counter()

    kx_min = 0.512 * np.sqrt(energy[-1]) * np.sin(theta[0]*np.pi/180)
    kx_max = 0.512 * np.sqrt(energy[-1]) * np.sin(theta[len(theta)-1]*np.pi/180)
    kx = np.linspace(kx_min, kx_max, len(theta))

    ky_min = 0.512 * np.sqrt(energy[-1]) * np.sin(phi[0]*np.pi/180)
    ky_max = 0.512 * np.sqrt(energy[-1]) * np.sin(phi[len(phi)-1]*np.pi/180)
    ky = np.linspace(ky_min, ky_max, len(phi))
  
    pool = Pool()
    input_array_tuple = zip(data, energy, repeat(theta), repeat(phi), repeat(kx), repeat(ky))
    data_k = pool.starmap(interp, input_array_tuple)
    pool.close()
    data_k = np.array(data_k)

    e_bin = fermi_energy - energy
    
    t_final = time.perf_counter()
    print("The program took", t_final - t0, "second(s).")
    return data_k, e_bin, kx, ky

In [9]:
data_k_mp_alt, e_bin, kx, ky = k_conv3d_mp_alt(data, energy, theta, phi, 16.7)

The program took 39.39358699400009 second(s).


In [10]:
np.array_equal(data_k_mp, data_k_mp_alt, equal_nan=True)

False

In [11]:
print(data_k_mp_alt[150][271][25])
data_k_mp[150][271][25]

181.98429096643224


181.98429096643216

In [12]:
np.array_equal(np.isnan(data_k_mp), np.isnan(data_k_mp_alt))

True